In [1]:
import numpy as np
import pandas as pd
import string
import os
from collections import Counter
import copy

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
pd.options.display.max_colwidth = 50
pd.set_option('display.max_rows', 500)
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
new_test=pd.read_csv('bases/new_test_variants.csv')
new_test_texts = pd.read_csv('bases/new_test_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
new_test_final=pd.merge(new_test,new_test_texts,how="left",on="ID")

In [3]:
leaks=pd.read_csv('bases/s1_add_train.csv')
leaks_1=pd.DataFrame([leaks["ID"],leaks.drop("ID",axis=1).idxmax(axis=1).map(lambda x: x.lstrip('class'))])
leaks_2=leaks_1.T
leaks_2.columns=["ID","Class"]

In [4]:
train = pd.read_csv('bases/training_variants')
test = pd.read_csv('bases/test_variants')

In [5]:
train_texts = pd.read_csv('bases/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
test_texts = pd.read_csv('bases/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")

In [6]:
train = pd.merge(train, train_texts, how='left', on='ID')
test = pd.merge(test, test_texts, how='left', on='ID')

In [7]:
leaks_3=pd.merge(leaks_2,test[test.ID.isin(leaks_2.ID)])
leaks_final=pd.merge(leaks_3,test_texts[test_texts.ID.isin(leaks_3.ID)])

In [8]:
leaks_final

,ID,Class,Gene,Variation,Text
0,12,1,TET2,Y1902A,TET proteins oxidize 5-methylcytosine (5mC) on...
1,19,2,MTOR,D2512H,Genes encoding components of the PI3K-Akt-mTOR...
2,21,2,KIT,D52N,Myeloproliferative disorders (MPD) constitute ...
3,55,4,SPOP,F125V,"In the largest E3 ligase subfamily, Cul3 binds..."
4,64,4,KEAP1,C23Y,Keap1 is the substrate recognition module of a...
5,70,4,CDKN2A,G35V,Inherited mutations affecting the INK4a/ARF lo...
6,89,9,IDH1,R132S,To assess the value of anti-isocitrate dehydro...
7,93,7,KIT,N822K,c-KIT is a member of the type 3 subclass of tr...
8,111,7,ETV1,Amplification,E26 transformation-specific (ETS) transcriptio...
9,113,7,JAK1,F958C,Activating mutations in JAK1 and JAK2 have bee...


In [124]:
merge_new = leaks_final.merge(merge_match, left_on=['Gene', 'Variation'], right_on = ['Gene', 'Variation'])

In [125]:
merge_new

,ID,Class_x,Gene,Variation,Text_x,ID_x,Class_y,ID_y,Text_y
0,12,1,TET2,Y1902A,TET proteins oxidize 5-methylcytosine (5mC) on...,202,1,12,TET proteins oxidize 5-methylcytosine (5mC) on...
1,19,2,MTOR,D2512H,Genes encoding components of the PI3K-Akt-mTOR...,123,2,19,Genes encoding components of the PI3K-Akt-mTOR...
2,21,2,KIT,D52N,Myeloproliferative disorders (MPD) constitute ...,760,2,21,Myeloproliferative disorders (MPD) constitute ...
3,55,4,SPOP,F125V,"In the largest E3 ligase subfamily, Cul3 binds...",147,4,55,"In the largest E3 ligase subfamily, Cul3 binds..."
4,64,4,KEAP1,C23Y,Keap1 is the substrate recognition module of a...,631,4,64,Keap1 is the substrate recognition module of a...
5,70,4,CDKN2A,G35V,Inherited mutations affecting the INK4a/ARF lo...,268,4,70,Inherited mutations affecting the INK4a/ARF lo...
6,89,9,IDH1,R132S,To assess the value of anti-isocitrate dehydro...,712,9,89,To assess the value of anti-isocitrate dehydro...
7,93,7,KIT,N822K,c-KIT is a member of the type 3 subclass of tr...,146,7,93,c-KIT is a member of the type 3 subclass of tr...
8,111,7,ETV1,Amplification,E26 transformation-specific (ETS) transcriptio...,809,7,111,E26 transformation-specific (ETS) transcriptio...
9,113,7,JAK1,F958C,Activating mutations in JAK1 and JAK2 have bee...,850,7,113,Activating mutations in JAK1 and JAK2 have bee...


In [126]:
Index_nothing = [item for item in list(leaks_final.ID) if item not in list(merge_new.ID)]

In [127]:
Index_nothing

[1623]

In [122]:
leaks_final

,ID,Class,Gene,Variation,Text
0,12,1,TET2,Y1902A,TET proteins oxidize 5-methylcytosine (5mC) on...
1,19,2,MTOR,D2512H,Genes encoding components of the PI3K-Akt-mTOR...
2,21,2,KIT,D52N,Myeloproliferative disorders (MPD) constitute ...
3,55,4,SPOP,F125V,"In the largest E3 ligase subfamily, Cul3 binds..."
4,64,4,KEAP1,C23Y,Keap1 is the substrate recognition module of a...
5,70,4,CDKN2A,G35V,Inherited mutations affecting the INK4a/ARF lo...
6,89,9,IDH1,R132S,To assess the value of anti-isocitrate dehydro...
7,93,7,KIT,N822K,c-KIT is a member of the type 3 subclass of tr...
8,111,7,ETV1,Amplification,E26 transformation-specific (ETS) transcriptio...
9,113,7,JAK1,F958C,Activating mutations in JAK1 and JAK2 have bee...


In [118]:
Index_nothing = [item for item in list(merge_new.ID_x) if item not in list(merge_match.ID_x)]

In [119]:
Index_nothing

[]

In [10]:
train_all = pd.concat([train,leaks_final]) #adding first stage

In [11]:
merge_match = new_test.merge(train_all, left_on=['Gene', 'Variation'], right_on = ['Gene', 'Variation'])
Index_leak = list(merge_match.ID_x - 1)
Index_no_leak = [item for item in new_test_final.index if item not in Index_leak]
test_no_leaks = new_test_final.iloc[Index_no_leak]
test_no_leaks

,ID,Gene,Variation,Text
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...
5,6,CHEK2,E239K,The nuclei that laboratories solution p53 KIT ...
6,7,CHST3,T141M,Myeloid differentiation 88 (MyD88) is the key ...
7,8,RNF6,G244D,Human ESCCs 2 occur frequently worldwide (1) ....
8,9,SPAST,C448Y,large were of activity growth this product tol...
10,11,SCN4A,V445M,Endometrial carcinoma is the most common gynec...
14,15,ERBB2,G746S,The protein-kinase family is the most frequent...
15,16,TP53,Y234S,Among the best-studied therapeutic targets in ...
16,17,RAB27A,A87P,"Introduction In recent years, a better unders..."


# Inserting the probs + the leak classes 

In [12]:
merge_match

,ID_x,Gene,Variation,Class,ID_y,Text
0,1,CHEK2,H371Y,4,3349,The incidence of breast cancer is increasing i...
1,2,AXIN2,Truncating Mutations,1,4609,An unselected series of 310 colorectal carcino...
2,5,BRAF,T599insTT,7,4631,Pilocytic astrocytoma (PA) is emerging as a tu...
3,10,AKT1,R328A,7,2362,The protein kinase v-akt murine thymoma viral ...
4,12,KIT,S709F,2,2771,Gastrointestinal stromal tumors (GIST) are cha...
5,13,BRCA2,V211I,1,5143,Although most BRCA sequence variants are clear...
6,14,ALK,F1174C,7,5199,"In the era of personalized medicine, understan..."
7,21,PTPN11,N308D,7,1843,Noonan syndrome is a developmental disorder wi...
8,24,AKT2,BCAM-AKT2 Fusion,7,5451,High-grade serous ovarian cancer (HGSC) is the...
9,26,MET,V1220I,7,1722,Point mutations emerge as one of the rate-limi...


In [13]:
# For the 366 cases create dummy of class
class_dummy = pd.get_dummies(merge_match.Class)
class_dummy

,1,2,3,4,5,6,7,8,9
0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,0,1,0,0


In [17]:
submission = pd.read_csv('stage2_sample_submission.csv')

In [18]:
# Inserting the dummies to the submission
submission.iloc[Index_leak, 1:] = class_dummy.values
submission

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0,0,0,1,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,0
5,6,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0
7,8,0,0,0,0,0,0,0,0,0
8,9,0,0,0,0,0,0,0,0,0
9,10,0,0,0,0,0,0,1,0,0


In [25]:
test_blind=submission[~submission.drop("ID",axis=1).any(axis=1)].replace(0,1/9)

In [26]:
shared_genes = list(set(train.Gene).intersection(set(test_no_leaks.Gene))) # the shared genes are those that appear in both
len(shared_genes)

35

In [28]:
probs_stacking_xgb = pd.read_csv('final_scores/new_stacked_test_xgb.csv')
probs_stacking_linear = pd.read_csv('final_scores/new_stacked_test_linear.csv')
#probs_backup_sub = pd.read_csv('final_scores/new_backup_sub.csv')

In [36]:
probs_stacking_xgb.shape

(619, 10)

In [40]:
late_sub=pd.concat([probs_stacking_xgb,test_blind]).groupby("ID").mean()

In [44]:
submission_late=submission[:]
submission_late.iloc[Index_no_leak, 1:] = late_sub.loc[:, :'class9'].values
submission_late
submission_late.to_csv("submission_late.csv", index=False)

In [ ]:
submission_xgb = submission[:]
submission_xgb.iloc[Index_no_leak, 1:] = probs_stacking_xgb.loc[:, :'class9'].values
submission_xgb

In [30]:
submission_xgb = submission[:]
submission_xgb.iloc[Index_no_leak, 1:] = probs_stacking_xgb.loc[:, :'class9'].values
submission_xgb

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,0.109587,0.190621,0.004193,0.053487,0.027290,0.039592,0.568215,0.004028,0.002988
3,4,0.098447,0.185763,0.006495,0.468194,0.013253,0.047157,0.115712,0.041593,0.023385
4,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
5,6,0.255913,0.041479,0.009485,0.598239,0.045022,0.024469,0.021054,0.002039,0.002300
6,7,0.127100,0.235374,0.028305,0.283178,0.021188,0.045214,0.199842,0.056523,0.003275
7,8,0.256782,0.028755,0.071392,0.409780,0.123698,0.076026,0.006991,0.002595,0.023981
8,9,0.146453,0.059563,0.004534,0.620152,0.021267,0.018442,0.124821,0.002859,0.001909
9,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [31]:
submission_linear = submission[:]
submission_linear.iloc[Index_no_leak, 1:] = probs_stacking_linear.loc[:, :'class9'].values
submission_linear

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,0.087939,0.182124,0.011494,0.067883,0.045370,0.045733,0.534100,0.012164,0.013192
3,4,0.133808,0.145251,0.017993,0.380675,0.045302,0.037045,0.164823,0.037654,0.037449
4,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
5,6,0.129109,0.028944,0.023530,0.697448,0.050758,0.032142,0.027827,0.004004,0.006239
6,7,0.145521,0.151305,0.019693,0.249540,0.042978,0.037111,0.252718,0.056463,0.044673
7,8,0.322214,0.041613,0.021670,0.441054,0.067661,0.051088,0.029379,0.009259,0.016063
8,9,0.154204,0.052751,0.012748,0.610105,0.034461,0.030402,0.081781,0.011374,0.012174
9,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [32]:
submission_backup_sub = submission[:]
submission_backup_sub.iloc[Index_no_leak, 1:] = probs_backup_sub.loc[:, :'class9'].values
submission_backup_sub

NameError: name 'probs_backup_sub' is not defined

In [101]:
submission_backup_sub[submission_backup_sub.class8>0.1]

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
67,68,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
299,300,0.049505,0.163048,0.009172,0.035936,0.033109,0.014948,0.492322,0.196971,0.004990
536,537,0.035727,0.038810,0.004573,0.100511,0.009997,0.005539,0.189502,0.183818,0.431525
946,947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [83]:
submission_linear.to_csv("submission_linear.csv", index=False)

In [85]:
submission_xgb.to_csv("submission_xgb.csv", index=False)

In [87]:
submission_backup_sub.to_csv("submission_backup_sub.csv", index=False)